In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [28]:
test_data = pd.read_csv('test.csv')
train_data = pd.read_csv('train.csv')

In [29]:
# plt.scatter(test_data.Square, test_data.KitchenSquare)

In [30]:
# Функции для очистки и подготовки данных
mean_year = np.round(train_data.loc[train_data['HouseYear'] <= 2020, 'HouseYear'].mean())
mean_healthcare = np.round(train_data["Healthcare_1"].mean())
mean_square_for_max = train_data.loc[(train_data['Rooms'] <= train_data.loc[(train_data['Square'] > 300), 'Rooms'].mean()), 'Square'].mean()
mean_square_for_big_ls = train_data.loc[train_data['LifeSquare'] > 250, 'Square'].mean()
mean_life_squae_for_max = train_data.loc[train_data['Square'] >= mean_square_for_big_ls, 'LifeSquare'].mean()
mean_square_Kitchen=train_data.KitchenSquare.mean()

In [31]:
def clean_year(df, mean_year):
    df.loc[df['HouseYear'] > 2020, 'HouseYear'] = mean_year

In [32]:
def clean_life_square(df, koef_S_LS):
    df.loc[(df['LifeSquare'] < 21) | (df['LifeSquare'].isnull()), 'LifeSquare'] = df['Square']*0.85
    df.loc[df['LifeSquare'] > 250, 'LifeSquare'] = mean_life_squae_for_max

In [33]:
def clean_square(df, mean_square_for_max):
    df.loc[(df['Square'] > 300), 'Square'] = mean_square_for_max

In [34]:
def clean_healthcare_1(df, mean_healthcare):
    df.loc[df['Healthcare_1'].isnull(), 'Healthcare_1'] = mean_healthcare

In [35]:
def clean_rooms(df):
    df.loc[(df['Rooms'] < 1) & (df['LifeSquare'] < 30), 'Rooms'] = 1
    df.loc[(df['Rooms'] < 1) & (df['LifeSquare'] > 30) & (df['LifeSquare'] < 45), 'Rooms'] = 2
    df.loc[(df['Rooms'] < 1) & (df['LifeSquare'] > 45) & (df['LifeSquare'] < 60), 'Rooms'] = 3
    df.loc[(df['Rooms'] < 1) & (df['LifeSquare'] > 60) & (df['LifeSquare'] < 75), 'Rooms'] = 4
    df.loc[(df['Rooms'] < 1) & (df['LifeSquare'] > 70), 'Rooms'] = 6
    df.loc[(df['Rooms'] > 10), 'Rooms'] = 2

In [38]:
def KitchenSquare(df, mean_square_for_max):
    df.loc[(df['KitchenSquare'] < 2) | (df['KitchenSquare'].isnull()), 'KitchenSquare'] = mean_square_Kitchen
    df.loc[(df['KitchenSquare'] > 90)] = mean_square_Kitchen

In [39]:
def prepare_data(df, mean_year=mean_year, mean_healthcare=mean_healthcare, 
                 mean_square_for_max=mean_square_for_max, mean_life_squae_for_max=mean_life_squae_for_max, 
                 mean_square_Kitchen=mean_square_Kitchen):
    clean_year(df, mean_year)
    clean_life_square(df, mean_life_squae_for_max)
    clean_healthcare_1(df, mean_healthcare)
    clean_rooms(df)
    clean_square(df, mean_square_for_max)
#     KitchenSquare(df, mean_square_Kitchen)

In [40]:
prepare_data(train_data)
prepare_data(test_data)

In [41]:
X = pd.get_dummies(train_data)
X.drop("Price", axis=1, inplace=True)
X.drop("Id", axis=1, inplace=True)
y = train_data.Price

In [42]:
train_data

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969.0,0.089040,B,B,33,7976,5,1143.0,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978.0,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968.0,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977.0,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976.0,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,77,32,2.0,50.401785,30.476203,5.0,6,5.0,1968.0,0.135650,B,B,46,7960,6,350.0,3,11,B,196684.316040
9996,6159,18,1.0,41.521546,35.293314,9.0,13,13.0,2000.0,0.000000,B,B,30,5562,0,1143.0,0,5,A,189050.289571
9997,5123,27,1.0,47.939008,40.748157,1.0,12,16.0,2015.0,0.072158,B,B,2,629,1,1143.0,0,0,A,159143.805370
9998,5400,75,2.0,43.602562,33.840147,8.0,1,5.0,1961.0,0.307467,B,A,30,5048,9,325.0,2,5,B,181595.339808


In [14]:
# df_num_features = train_data.select_dtypes(include=['float64','uint8'])  # отбираем количественные признаки
# import seaborn as sns
# sns.pairplot(df_num_features);

# GradientBoostingRegressor

In [43]:
test_data = pd.read_csv('test.csv')
train_data = pd.read_csv('train.csv')
prepare_data(train_data)
prepare_data(test_data)
X = pd.get_dummies(train_data)
X.drop("Price", axis=1, inplace=True)
X.drop("Id", axis=1, inplace=True)
y = train_data.Price

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.16, random_state=42)

In [44]:

# переобучение и оценка модели
from sklearn.ensemble import GradientBoostingRegressor
final_model = GradientBoostingRegressor(n_estimators=200, max_depth=5, random_state=42
                                       )
# min_samples_split=5, subsample=0.5 , min_samples_leaf=4 

final_model.fit(X_train, y_train)

y_pred_gbr = final_model.predict(X_valid)
y_pred_train_gbr = final_model.predict(X_train)

r2_score(y_valid, y_pred_gbr)

0.7577861874526557

In [26]:
k=50
from sklearn.ensemble import GradientBoostingRegressor
for number in range(15):
    k=round(k+5,2)
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.16, random_state=42)
    
    # переобучение и оценка модели
    def clean_life_square(df, koef_S_LS):
        df.loc[(df['LifeSquare'] < 21) | (df['LifeSquare'].isnull()), 'LifeSquare'] = df['Square']*0.5
        df.loc[df['LifeSquare'] > 250, 'LifeSquare'] = mean_life_squae_for_max
    
    def KitchenSquare(df, mean_square_for_max):
        df.loc[(df['KitchenSquare'] < 3) | (df['KitchenSquare'].isnull()), 'KitchenSquare'] = mean_square_Kitchen
        df.loc[(df['KitchenSquare'] > k)] = mean_square_Kitchen
    
    def clean_square(df, mean_square_for_max):
        df.loc[(df['Square'] > 300), 'Square'] = mean_square_for_max


    
    test_data = pd.read_csv('test.csv')
    train_data = pd.read_csv('train.csv')
    prepare_data(train_data)
    prepare_data(test_data)
    X = pd.get_dummies(train_data)
    X.drop("Price", axis=1, inplace=True)
    X.drop("Id", axis=1, inplace=True)
    y = train_data.Price
#     print(train_data.LifeSquare.sum())
    
    final_model = GradientBoostingRegressor(n_estimators=200, max_depth=5, random_state=42
                                           )
    # min_samples_split=5, subsample=0.5 , min_samples_leaf=4 

    final_model.fit(X_train, y_train)

    y_pred_gbr = final_model.predict(X_valid)
    y_pred_train_gbr = final_model.predict(X_train)

    print('r2: ', r2_score(y_valid, y_pred_gbr),', KitchenSquare: ',k)

r2:  0.7561424262489411 , KitchenSquare:  55
r2:  0.7525446791425944 , KitchenSquare:  60
r2:  0.7537028187364678 , KitchenSquare:  65
r2:  0.763877793442332 , KitchenSquare:  70
r2:  0.7630545223828556 , KitchenSquare:  75
r2:  0.7515848449232532 , KitchenSquare:  80
r2:  0.7603152394822067 , KitchenSquare:  85
r2:  0.7561317619495076 , KitchenSquare:  90
r2:  0.7561317619495076 , KitchenSquare:  95
r2:  0.7561317619495076 , KitchenSquare:  100
r2:  0.7527481824489445 , KitchenSquare:  105
r2:  0.7527481824489445 , KitchenSquare:  110
r2:  0.7527481824489445 , KitchenSquare:  115
r2:  0.7498191791217641 , KitchenSquare:  120
r2:  0.7498191791217641 , KitchenSquare:  125


0.7621056876187297 - test_size=0.16 - n_estimators=200, max_depth=5, random_state=42 (0.75339)

KitchenSquare min 3,5,8
r2:  0.763877793442332 , KitchenSquare:  70
r2:  0.7630545223828556 , KitchenSquare:  75
r2:  0.7603152394822067 , KitchenSquare:  85
r2:  0.7561317619495076 , KitchenSquare:  100

In [ ]:
X = pd.get_dummies(train_data)
X.drop("Price", axis=1, inplace=True)
X.drop("Id", axis=1, inplace=True)
y = train_data.Price

In [ ]:
# Предсказываем цены для тестовых данных и выгружаем в файл
X_test = pd.get_dummies(test_data)
X_test.drop("Id", axis=1, inplace=True)
test_data["Price"] = final_model.predict(X_test)

In [44]:
# экспорт в файл
test_data.loc[:, ['Id', 'Price']].to_csv('best_gbr_01.csv', index=False)

In [71]:
from sklearn.metrics import r2_score as r2, mean_absolute_error as mae, mean_squared_error as mse
import seaborn as sns
def evaluate_preds(true_values, pred_values):
    print("R2:\t" + str(round(r2(true_values, pred_values), 9)) + "\n" +
          "MAE:\t" + str(round(mae(true_values, pred_values), 9)) + "\n" +
          "MSE:\t" + str(round(mse(true_values, pred_values), 9)))
    
    plt.figure(figsize=(10,10)) 
    
    sns.scatterplot(x=pred_values, y=true_values)
    
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('True vs Predicted values')
    plt.show()

In [76]:
# y_train_preds = final_model.predict(X_train)
# evaluate_preds(y_train, y_train_preds)